# Experiment Results Loader

A lightweight script to discover, load and display experiment configurations and results using Rich and Pandas.

## Features

- **Directory Discovery**  
  Enumerates experiment folders under a base `experiments/` path.

- **Config Preview**  
  Renders each experiment’s `config.yaml` with syntax highlighting in the console.

- **Results Import**  
  Reads `results.xlsx`, organizes sheets by variant and metric, and loads into DataFrames.

- **Sorting by SMAPE**  
  Automatically sorts result tables by one of the SMAPE columns (`SMAPE_VAL`, `SMAPE_TEST`, `SMAPE`).

- **Stylized Output**  
  Shows the top 10 rows of key metrics with custom styling via `DataFrameStylerAuto`.

In [1]:
import logging
from pathlib import Path
from typing import Dict

import pandas as pd
from rich.console import Console
from rich.syntax import Syntax
from rich.panel import Panel
from IPython.display import display
import re
from typing import Dict, Callable, Optional


from statistical.statistical_framework import DataFrameStylerAuto
from utils.utilities import print_style
from themes import themes

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
logger = logging.getLogger(__name__)
console = Console()

# Constants
DEFAULT_VARIANT = "default"
SORT_PRIORITY_COLUMNS = ["SMAPE_VAL", "SMAPE_TEST", "SMAPE"]

In [2]:
def list_experiment_paths(base_dir: Path) -> Dict[int, Path]:
    """
    List experiment directories under the given base directory.

    Args:
        base_dir (Path): Base path to the 'experiments' directory.

    Returns:
        Dict[int, Path]: Mapping from numerical index to experiment path.

    Raises:
        FileNotFoundError: If the base directory does not exist.
    """
    if not base_dir.is_dir():
        raise FileNotFoundError(f"Experiments directory '{base_dir}' does not exist.")

    experiment_paths = sorted(
        [p for p in base_dir.iterdir() if p.is_dir() and not p.name.startswith(".")],
        key=lambda p: p.name
    )
    return {index: path for index, path in enumerate(experiment_paths)}


def display_experiment_details(exp_path: Path) -> None:
    """
    Display the 'config.yaml' file for the specified experiment using Rich.

    Args:
        exp_path (Path): Path to the experiment directory.
    """
    config_file = exp_path / "config.yaml"
    if not config_file.is_file():
        console.print(f"[red]Missing config.yaml in {exp_path}[/red]")
        return

    config_text = config_file.read_text()
    syntax = Syntax(config_text, "yaml", theme="ansi_light", line_numbers=True)
    console.print(Panel(syntax, title=f"[bold green]Experiment: {exp_path.name}"))


def load_organized_results(exp_path: Path) -> Dict[str, Dict[str, pd.DataFrame]]:
    """
    Load and organize results from an Excel file in the experiment directory.

    Args:
        exp_path (Path): Path to the experiment directory.

    Returns:
        Dict[str, Dict[str, pd.DataFrame]]: Nested mapping of variant to metric name to DataFrame.
    """
    excel_file = exp_path / "results.xlsx"
    if not excel_file.is_file():
        console.print(f"[red]Missing results.xlsx in {exp_path}[/red]")
        return {}

    xls = pd.ExcelFile(excel_file)
    organized: Dict[str, Dict[str, pd.DataFrame]] = {}
    for sheet_name in xls.sheet_names:
        variant, metric = sheet_name.split("__", 1) if "__" in sheet_name else (DEFAULT_VARIANT, sheet_name)
        organized.setdefault(variant, {})[metric] = xls.parse(sheet_name)
    return organized


def load_all_experiments(base_dir: Path) -> Dict[str, Dict[str, Dict[str, pd.DataFrame]]]:
    """
    Load and display all experiments in the base directory.

    Args:
        base_dir (Path): Path to the 'experiments' directory.

    Returns:
        Dict[str, Dict[str, Dict[str, pd.DataFrame]]]: Mapping from experiment name to organized results.
    """
    experiment_paths = list_experiment_paths(base_dir)
    all_results: Dict[str, Dict[str, Dict[str, pd.DataFrame]]] = {}

    for path in experiment_paths.values():
        display_experiment_details(path)
        organized = load_organized_results(path)
        if organized:
            all_results[path.name] = organized

    console.print(f"\n[bold green]Loaded {len(all_results)} experiments.[/bold green]")
    return all_results


def sort_results_by_smape(results_by_variant: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
    """
    Sort each DataFrame of results by the SMAPE column, if present.

    Args:
        results_by_variant (Dict[str, pd.DataFrame]): Mapping of metric names to DataFrames.

    Returns:
        Dict[str, pd.DataFrame]: Sorted DataFrames by metric.
    """
    sorted_variants: Dict[str, pd.DataFrame] = {}
    for metric_name, df in results_by_variant.items():
        if not isinstance(df, pd.DataFrame):
            continue
        sort_col = next((col for col in SORT_PRIORITY_COLUMNS if col in df.columns), None)
        df_sorted = df.sort_values(sort_col, ascending=True).reset_index(drop=True) if sort_col else df.copy()
        sorted_variants[metric_name] = df_sorted
    return sorted_variants


def main() -> None:
    """
    Main entry point to load, process, and display experiment results.
    """
    base_dir = Path.cwd().parents[1] / "experiments"
    experiments = load_all_experiments(base_dir)
    for experiment_name, variants in experiments.items():
        print_style(experiment_name)
        default_results = variants.get(DEFAULT_VARIANT, {})
        sorted_results = sort_results_by_smape(default_results)
        for metric_key in ("cumulative_metrics", "aggregated_metrics"):
            df = sorted_results.get(metric_key)
            if df is not None and not df.empty:
                head_df = df.head(10)
                styled = DataFrameStylerAuto.style_dataframe(head_df, "minimal", themes=themes)
                display(styled)
    logging.info("Script finished.")


if __name__ == "__main__":
    main()

╭──────────────────────────── Experiment: U-N-I-S-I-M-_-I-V_Seq2Context_ens1_20250702 ────────────────────────────╮
│    1 DEFAULT_EXP_PARAMS:                                                                                        │
│    2   aggregation_method: median                                                                               │
│    3   aggregation_quantiles: !!python/tuple                                                                    │
│    4   - 0.25                                                                                                   │
│    5   - 0.5                                                                                                    │
│    6   - 0.75                                                                                                   │
│    7   architecture_name: Seq2Context                                                                           │
│    8   band: null                                                                                               │
│    9   batch_size: 8                                                                                            │
│   10   epochs: 250                                                                                              │
│   11   evaluate_by_slice: true                                                                                  │
│   12   feature_kind: Normal                                                                                     │
│   13   horizon: 30                                                                                              │
│   14   lag_window: 30                                                                                           │
│   15   patience: 100                                                                                            │
│   16   plot: true                                                                                               │
│   17   scenario: P50                                                                                            │
│   18   show_components: false                                                                                   │
│   19   slice_ratios: !!python/tuple                                                                             │
│   20   - 1.0                                                                                                    │
│   21   test_size: 0.6                                                                                           │
│   22   use_known_good: false                                                                                    │
│   23 datasets_to_run: UNISIM_IV                                                                                 │
│   24 num_ensemble_models: 1                                                                                     │
│   25                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────── Experiment: U-N-I-S-I-M-_-I-V_Seq2PIN_ens1_20250702 ──────────────────────────────╮
│    1 DEFAULT_EXP_PARAMS:                                                                                        │
│    2   aggregation_method: median                                                                               │
│    3   aggregation_quantiles: !!python/tuple                                                                    │
│    4   - 0.25                                                                                                   │
│    5   - 0.5                                                                                                    │
│    6   - 0.75                                                                                                   │
│    7   architecture_name: Seq2PIN                                                                               │
│    8   band: null                                                                                               │
│    9   batch_size: 8                                                                                            │
│   10   epochs: 250                                                                                              │
│   11   evaluate_by_slice: true                                                                                  │
│   12   feature_kind: Normal                                                                                     │
│   13   horizon: 30                                                                                              │
│   14   lag_window: 30                                                                                           │
│   15   patience: 100                                                                                            │
│   16   plot: true                                                                                               │
│   17   scenario: P50                                                                                            │
│   18   show_components: false                                                                                   │
│   19   slice_ratios: !!python/tuple                                                                             │
│   20   - 1.0                                                                                                    │
│   21   test_size: 0.6                                                                                           │
│   22   use_known_good: false                                                                                    │
│   23 datasets_to_run: UNISIM_IV                                                                                 │
│   24 num_ensemble_models: 1                                                                                     │
│   25                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────── Experiment: UN_IV_P11_E3_RePINN_ALL ──────────────────────────────────────╮
│    1 DEFAULT_EXP_PARAMS:                                                                                        │
│    2   aggregation_method: median                                                                               │
│    3   aggregation_quantiles: !!python/tuple                                                                    │
│    4   - 0.25                                                                                                   │
│    5   - 0.5                                                                                                    │
│    6   - 0.75                                                                                                   │
│    7   architecture_name: Seq2Context                                                                           │
│    8   band: null                                                                                               │
│    9   batch_size: 8                                                                                            │
│   10   epochs: 250                                                                                              │
│   11   evaluate_by_slice: true                                                                                  │
│   12   feature_kind: Normal                                                                                     │
│   13   horizon: 30                                                                                              │
│   14   lag_window: 30                                                                                           │
│   15   patience: 50                                                                                             │
│   16   plot: false                                                                                              │
│   17   scenario: P50                                                                                            │
│   18   show_components: false                                                                                   │
│   19   slice_ratios: !!python/tuple                                                                             │
│   20   - 1.0                                                                                                    │
│   21   test_size: 0.6                                                                                           │
│   22   use_known_good: false                                                                                    │
│   23 datasets_to_run: UNISIM_IV                                                                                 │
│   24 num_ensemble_models: 3                                                                                     │
│   25                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────── Experiment: UN_IV_P11_E3_Trend ─────────────────────────────────────────╮
│    1 DEFAULT_EXP_PARAMS:                                                                                        │
│    2   aggregation_method: median                                                                               │
│    3   aggregation_quantiles: !!python/tuple                                                                    │
│    4   - 0.25                                                                                                   │
│    5   - 0.5                                                                                                    │
│    6   - 0.75                                                                                                   │
│    7   architecture_name: Seq2Trend                                                                             │
│    8   band: null                                                                                               │
│    9   batch_size: 8                                                                                            │
│   10   epochs: 250                                                                                              │
│   11   evaluate_by_slice: true                                                                                  │
│   12   feature_kind: Normal                                                                                     │
│   13   horizon: 30                                                                                              │
│   14   lag_window: 30                                                                                           │
│   15   patience: 100                                                                                            │
│   16   plot: false                                                                                              │
│   17   scenario: P50                                                                                            │
│   18   show_components: false                                                                                   │
│   19   slice_ratios: !!python/tuple                                                                             │
│   20   - 1.0                                                                                                    │
│   21   test_size: 0.6                                                                                           │
│   22   use_known_good: false                                                                                    │
│   23 datasets_to_run: UNISIM_IV                                                                                 │
│   24 num_ensemble_models: 3                                                                                     │
│   25                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────── Experiment: UN_IV_P11_PINN ───────────────────────────────────────────╮
│    1 DEFAULT_EXP_PARAMS:                                                                                        │
│    2   aggregation_method: median                                                                               │
│    3   aggregation_quantiles: !!python/tuple                                                                    │
│    4   - 0.25                                                                                                   │
│    5   - 0.5                                                                                                    │
│    6   - 0.75                                                                                                   │
│    7   architecture_name: Seq2PIN                                                                               │
│    8   band: null                                                                                               │
│    9   batch_size: 8                                                                                            │
│   10   epochs: 250                                                                                              │
│   11   evaluate_by_slice: true                                                                                  │
│   12   feature_kind: Normal                                                                                     │
│   13   horizon: 30                                                                                              │
│   14   lag_window: 30                                                                                           │
│   15   patience: 50                                                                                             │
│   16   plot: true                                                                                               │
│   17   scenario: P50                                                                                            │
│   18   show_components: false                                                                                   │
│   19   slice_ratios: !!python/tuple                                                                             │
│   20   - 1.0                                                                                                    │
│   21   test_size: 0.6                                                                                           │
│   22   use_known_good: false                                                                                    │
│   23 datasets_to_run: UNISIM_IV                                                                                 │
│   24 num_ensemble_models: 1                                                                                     │
│   25                                                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

DataFrame shape: 10 rows x 11 columns; Index range: 0 to 9


,Well,Category,strategy,extractor,fuser,R²_VAL,MAE_VAL,SMAPE_VAL,R²_TEST,MAE_TEST,SMAPE_TEST
0,15/9-F-14,Cumulative,pressure_ensemble,identity,bias_scale,0.99,30511.34,0.16,0.96,213105.26,0.97
1,15/9-F-14,Cumulative,pressure_ensemble,aggregate,bias_scale,0.99,36046.41,0.19,0.97,210629.22,0.96
2,15/9-F-14,Cumulative,combined_exp_arps,identity,bias_scale,0.96,73270.80,0.38,0.78,541986.09,2.45
3,15/9-F-14,Cumulative,pressure_ensemble,rnn,bias_scale,0.95,76884.65,0.40,0.97,212096.78,0.97
4,15/9-F-14,Cumulative,arps,identity,bias_scale,0.95,77094.15,0.40,0.79,533493.83,2.41
5,15/9-F-14,Cumulative,pressure_ensemble,cnn,bias_scale,0.94,82544.95,0.43,0.97,197736.39,0.90
6,15/9-F-14,Cumulative,exponential,identity,bias_scale,0.94,85773.29,0.45,0.68,650557.77,2.93
7,15/9-F-14,Cumulative,pressure_ensemble,tcn,bias_scale,0.94,89351.61,0.46,0.97,204252.31,0.93
8,15/9-F-14,Cumulative,exponential,aggregate,bias_scale,0.93,92056.01,0.48,0.79,531706.44,2.41
9,15/9-F-14,Cumulative,arps,aggregate,bias_scale,0.92,97347.66,0.50,0.77,558214.68,2.53


DataFrame shape: 10 rows x 11 columns; Index range: 0 to 9


,Well,Category,strategy,extractor,fuser,R²_VAL,MAE_VAL,SMAPE_VAL,R²_TEST,MAE_TEST,SMAPE_TEST
0,15/9-F-14,Aggregated,pressure_ensemble,tcn,bias_scale,-0.45,1829.13,22.72,0.80,497.90,15.85
1,15/9-F-14,Aggregated,pressure_ensemble,rnn,bias_scale,-0.46,1898.18,23.79,0.80,518.42,17.00
2,15/9-F-14,Aggregated,pressure_ensemble,cnn,bias_scale,-0.46,1902.07,23.84,0.79,518.24,16.47
3,15/9-F-14,Aggregated,pressure_ensemble,aggregate,bias_scale,-0.51,1858.78,24.13,0.78,570.66,18.83
4,15/9-F-14,Aggregated,exponential,tcn,bias_scale,-0.89,2281.64,27.20,0.73,679.77,20.10
5,15/9-F-14,Aggregated,arps,tcn,bias_scale,-0.92,2317.25,27.55,0.73,678.16,20.08
6,15/9-F-14,Aggregated,combined_exp_arps,tcn,bias_scale,-0.90,2320.45,27.65,0.73,683.34,19.97
7,15/9-F-14,Aggregated,exponential,rnn,bias_scale,-0.91,2415.23,28.94,0.72,714.97,21.73
8,15/9-F-14,Aggregated,combined_exp_arps,aggregate,bias_scale,-0.87,2352.16,28.99,0.68,811.90,24.74
9,15/9-F-14,Aggregated,arps,rnn,bias_scale,-0.91,2421.46,29.05,0.72,715.89,21.75


2025-07-02 16:51:41,418 [INFO] Script finished.


In [2]:
# # Transformation registry
# TRANSFORM_REGISTRY: Dict[str, Callable[[str], str]] = {
#     "phrase": lambda s: " ".join(w.capitalize() for w in re.sub(r"[_\-]+", " ", s).split()),
#     "upper": str.upper,
#     "lower": str.lower,
#     "sentence": lambda s: re.sub(r"[_\-]+", " ", s).capitalize(),
#     "capitalize": lambda s: s.replace("_", " ").replace("-", " ").title(),
#     "replacesymbol": lambda s: s.replace("_", " ").replace("-", " ")
# }


# def apply_string_transformation(value: object, transform_name: str) -> object:
#     """
#     Apply a named string transformation to `value` if it's a string, else return it unchanged.
#     """
#     if pd.isna(value):
#         return value
#     s = str(value)
#     func = TRANSFORM_REGISTRY.get(transform_name.strip().lower())
#     if not func:
#         raise ValueError(f"Unknown transformation: {transform_name!r}")
#     return func(s)


# def list_experiment_paths(base_dir: Path) -> Dict[int, Path]:
#     """
#     List experiment subdirectories under `base_dir`.
#     """
#     if not base_dir.is_dir():
#         raise FileNotFoundError(f"Experiments directory '{base_dir}' not found.")
#     dirs = sorted(
#         [p for p in base_dir.iterdir() if p.is_dir() and not p.name.startswith(".")],
#         key=lambda p: p.name
#     )
#     return {i: p for i, p in enumerate(dirs)}


# def display_experiment_config(exp_path: Path) -> None:
#     """
#     Display the contents of config.yaml in the experiment directory using Rich.
#     """
#     config_file = exp_path / "config.yaml"
#     if not config_file.is_file():
#         console.print(f"[red]Missing config.yaml in {exp_path}[/red]")
#         return
#     text = config_file.read_text()
#     syntax = Syntax(text, "yaml", theme="ansi_light", line_numbers=True)
#     console.print(Panel(syntax, title=f"[bold green]Experiment: {exp_path.name}"))


# def load_organized_results(exp_path: Path) -> Dict[str, Dict[str, pd.DataFrame]]:
#     """
#     Load an experiment's results.xlsx and return nested dict variant->metric->DataFrame.
#     """
#     excel_file = exp_path / "results.xlsx"
#     if not excel_file.is_file():
#         console.print(f"[red]Missing results.xlsx in {exp_path}[/red]")
#         return {}
#     xls = pd.ExcelFile(excel_file)
#     organized: Dict[str, Dict[str, pd.DataFrame]] = {}
#     for sheet in xls.sheet_names:
#         variant, metric = sheet.split("__", 1) if "__" in sheet else (DEFAULT_VARIANT, sheet)
#         organized.setdefault(variant, {})[metric] = xls.parse(sheet)
#     return organized


# def sort_results_by_smape(variant_results: Dict[str, pd.DataFrame]) -> Dict[str, pd.DataFrame]:
#     """
#     Sort each DataFrame by the first available SMAPE column ascending.
#     """
#     sorted_dict: Dict[str, pd.DataFrame] = {}
#     for metric, df in variant_results.items():
#         if not isinstance(df, pd.DataFrame):
#             continue
#         sort_col = next((c for c in SORT_PRIORITY_COLUMNS if c in df.columns), None)
#         sorted_dict[metric] = (
#             df.sort_values(sort_col, ascending=True)
#               .reset_index(drop=True)
#             if sort_col else df.copy()
#         )
#     return sorted_dict


# def preprocess_dataframe_for_latex(
#     df: pd.DataFrame,
#     column_transformations: Dict[str, str]
# ) -> pd.DataFrame:
#     """
#     Apply string transformations and rename columns; round numeric columns.
#     Use '*' key to apply transformation to all string columns.
#     """
#     if not isinstance(df, pd.DataFrame):
#         raise TypeError("df must be a pandas DataFrame.")
#     transforms = dict(column_transformations)
#     result = df.copy()

#     # Wildcard transformation
#     if "*" in transforms:
#         default = transforms.pop("*")
#         for col in result.select_dtypes(include=["object", "string"]).columns:
#             transforms.setdefault(col, default)

#     # Apply transformations to values
#     for col, trans in transforms.items():
#         if col not in result.columns:
#             logger.warning("Column '%s' not found; skipping.", col)
#             continue
#         result[col] = result[col].map(lambda v: apply_string_transformation(v, trans))

#     # Rename columns based on transformation
#     rename_map = {
#         col: apply_string_transformation(col, trans)
#         for col, trans in transforms.items()
#         if apply_string_transformation(col, trans) != col
#     }
#     result.rename(columns=rename_map, inplace=True)

#     # Round numeric columns
#     numeric_cols = result.select_dtypes(include="number").columns
#     result[numeric_cols] = result[numeric_cols].round(2)

#     return result


# def drop_columns(df: pd.DataFrame, cols: list) -> pd.DataFrame:
#     """
#     Return a copy of df without the specified columns.
#     """
#     return df.drop(columns=cols, errors="ignore")


# def filter_top_group(
#     df: pd.DataFrame,
#     group_col: str,
#     metric_col: str,
#     n_best: int = 3,
#     kind: str = "minor"
# ) -> pd.DataFrame:
#     """
#     For each value in `group_col`, keep the top `n_best` rows by `metric_col`.
#     `kind='minor'` → lowest values; `'major'` → highest.
#     """
#     ascending = kind.lower().startswith("minor")
#     sorted_df = df.sort_values([group_col, metric_col], ascending=[True, ascending])
#     top_df = sorted_df.groupby(group_col, group_keys=False).head(n_best)
#     return top_df.sort_values(metric_col, ascending=True).reset_index(drop=True)


# def escape_latex(text: str) -> str:
#     """
#     Escape LaTeX special characters in a string.
#     """
#     replacements = {
#         "&": r"\&", "%": r"\%", "$": r"\$", "#": r"\#",
#         "{": r"\{", "}": r"\}", "~": r"\textasciitilde{}",
#         "^": r"\textasciicircum{}", "\\": r"\textbackslash{}"
#     }
#     for old, new in replacements.items():
#         text = text.replace(old, new)
#     return text


# def dataframe_to_latex(
#     df: pd.DataFrame,
#     caption: str,
#     label: str,
#     alignment: Optional[str] = None,
#     float_format: str = "%.2f",
#     na_rep: str = "-",
#     escape_text: bool = True,
#     escape_header: bool = True,
#     index: bool = False,
#     header: bool = True,
#     bold_header: bool = True,
#     use_booktabs: bool = True
# ) -> str:
#     """
#     Convert a DataFrame to a LaTeX table string.
#     """
#     df_work = df.copy()
#     if index:
#         df_work = df_work.reset_index()
#         idx_name = df.index.name or "Index"
#         df_work.rename(columns={df_work.columns[0]: idx_name}, inplace=True)

#     cols = list(df_work.columns)
#     n_cols = len(cols)
#     align_spec = alignment if alignment and len(alignment) == n_cols else "l" + "c" * (n_cols - 1)

#     lines = [
#         r"\begin{table}[h!]",
#         r"  \centering",
#         f"  \\caption{{{caption}}}",
#         f"  \\label{{{label}}}",
#         r"  \begin{tabular}{@{}" + align_spec + r"@{}}",
#         r"  \toprule" if use_booktabs else r"  \hline"
#     ]

#     if header:
#         header_cells = []
#         for col in cols:
#             cell = escape_latex(col) if escape_header else col
#             header_cells.append(f"\\textbf{{{cell}}}" if bold_header else cell)
#         lines += [
#             "  " + " & ".join(header_cells) + r" \\",
#             r"  \midrule" if use_booktabs else r"  \hline"
#         ]

#     for _, row in df_work.iterrows():
#         row_cells = []
#         for val in row:
#             if pd.isna(val):
#                 cell = na_rep
#             elif isinstance(val, float):
#                 cell = float_format % val
#                 cell = escape_latex(cell) if escape_text else cell
#             else:
#                 cell = escape_latex(str(val)) if escape_text else str(val)
#             row_cells.append(cell)
#         lines.append("  " + " & ".join(row_cells) + r" \\")
#     lines += [
#         r"  \bottomrule" if use_booktabs else r"  \hline",
#         r"  \end{tabular}",
#         r"\end{table}"
#     ]
#     return "\n".join(lines)


# def main() -> None:
#     """
#     Main script to load, process, display, and export experiment results.
#     """
#     base_dir = Path.cwd().parents[1] / "experiments"
#     experiment_paths = list_experiment_paths(base_dir)
#     all_results: Dict[str, Dict[str, Dict[str, pd.DataFrame]]] = {}

#     for exp_path in experiment_paths.values():
#         display_experiment_config(exp_path)
#         organized = load_organized_results(exp_path)
#         if organized:
#             all_results[exp_path.name] = organized

#     console.print(f"\n[bold green]Loaded {len(all_results)} experiments.[/bold green]")

#     for exp_name, variants in all_results.items():
#         print_style(exp_name)
#         default_results = variants.get(DEFAULT_VARIANT, {})
#         sorted_default = sort_results_by_smape(default_results)

#         # Display top-10 tables for key metrics
#         for metric_key in ("cumulative_metrics", "aggregated_metrics"):
#             df_metric = sorted_default.get(metric_key)
#             if df_metric is not None and not df_metric.empty:
#                 styled = DataFrameStylerAuto.style_dataframe(df_metric.head(10), "minimal", themes)
#                 display(styled)

#         # Preprocess and export to LaTeX
#         transformations = {
#             "Well": "Upper",
#             "Category": "Phrase",
#             "strategy": "Phrase",
#             "extractor": "Upper",
#             "fuser": "Capitalize",
#             "*": "Capitalize"
#         }
#         df_cum = sorted_default.get("cumulative_metrics", pd.DataFrame()).head(100)
#         df_pre = preprocess_dataframe_for_latex(df_cum, transformations)
#         df_pre = drop_columns(df_pre, ["Well", "Category"])
#         df_top = filter_top_group(df_pre, group_col="Strategy", metric_col="SMAPE_TEST", n_best=3, kind="minor")

#         console.print("\n--- Preprocessed DataFrame ---")
#         console.print(df_top.to_string())

#         latex_code = dataframe_to_latex(
#             df_top,
#             caption="Cumulative Forecast Result at Well 15/9-F-14",
#             label="F_14_Cumulative",
#             escape_text=True
#         )
#         console.print("\n--- LaTeX Code ---")
#         console.print(latex_code)

#     logger.info("Script finished.")


# if __name__ == "__main__":
#     main()


In [70]:
import re
from textwrap import dedent
from typing import List, Tuple


def _get_tabular_content(latex_str: str) -> str:
    """Extract content inside the tabular environment."""
    start_tok = r"\\begin{tabular}"
    end_tok = r"\\end{tabular}"
    start = latex_str.find(start_tok)
    if start < 0:
        raise ValueError("Missing \"\\begin{tabular}\"")

    # find end of column spec
    spec_start = latex_str.find("{", start + len(start_tok))
    if spec_start < 0:
        raise ValueError("Malformed begin: missing '{'")
    idx = spec_start + 1
    depth = 1
    while idx < len(latex_str) and depth:
        if latex_str[idx] == '{':
            depth += 1
        elif latex_str[idx] == '}':
            depth -= 1
        idx += 1
    if depth:
        raise ValueError("Unbalanced braces in column spec")

    end = latex_str.find(end_tok, idx)
    if end < 0:
        raise ValueError("Missing \"\\end{tabular}\"")

    return latex_str[idx:end].strip()


def _clean_lines(raw: str) -> List[str]:
    """Filter out rules/comments and strip line endings."""
    lines: List[str] = []
    for line in raw.splitlines():
        text = line.strip()
        if not text or text.startswith(("\\toprule", "\\midrule", "\\bottomrule", "%")):
            continue
        if text.endswith('\\\\'):
            text = text[:-2].strip()
        lines.append(text)
    if not lines:
        raise ValueError("No table lines found")
    return lines


def extract_header_and_rows(latex_str: str) -> Tuple[str, List[str]]:
    """Get header and data rows from a LaTeX table string."""
    content = _get_tabular_content(latex_str)
    lines = _clean_lines(content)
    header, *rows = lines
    if not header:
        raise ValueError("Empty header row")
    return header, rows


def combine_tables_side_by_side(
    table1: str,
    table2: str,
    label1: str,
    label2: str,
    caption: str,
    label: str,
    cols_per: int = 6,
) -> str:
    """Merge two LaTeX tables side by side under one float."""
    hdr1, rows1 = extract_header_and_rows(table1)
    hdr2, rows2 = extract_header_and_rows(table2)

    if hdr1 != hdr2:
        # use first header if they differ
        hdr2 = hdr1

    if len(rows1) != len(rows2):
        raise ValueError(
            f"Row count mismatch: {len(rows1)} != {len(rows2)}"
        )

    spec = '@{}lllccc@{\quad}lllccc@{}'
    head_line = (
        f"\\multicolumn{{{cols_per}}}{{c}}{{\\textbf{{{label1}}}}} & "
        f"\\multicolumn{{{cols_per}}}{{c}}{{\\textbf{{{label2}}}}} \\\\"
    )
    col_line = f"{hdr1} & {hdr2} \\\\"

    data_lines = []
    for r1, r2 in zip(rows1, rows2):
        data_lines.append(f"  {r1} & {r2} \\\\")
    data_block = "\n".join(data_lines)

    table = dedent(f"""
    \begin{{table}}[h!]
      \centering
      \caption{{{caption}}}
      \label{{{label}}}
      \resizebox{{\textwidth}}{{!}}{{%
        \begin{{tabular}}{{{spec}}}
          \toprule
          {head_line}
          \midrule
          {col_line}
          \midrule
{data_block}
          \bottomrule
        \end{{tabular}} %
      }}
    \end{{table}}
    """
    )
    return table.strip()


# --- Example Usage ---
if __name__ == "__main__":


    # Combine the tables
    combined_table = combine_latex_tables_side_by_side(
        latex_table1_str=latex_code,
        latex_table2_str=latex_code,
        well1_name="15/9-F-14",
        well2_name="15/9-F-12",
        combined_caption="Side-by-Side Cumulative Forecast Results for Wells F-14 and F-12",
        combined_label="F-14_F-12_Cumulative_Combined",
        num_cols_per_table=6 # Strategy, EXTRACTOR, Fuser, R², MAE, SMAPE
    )

    print("\n--- Combined LaTeX Table ---")
    print(combined_table)


--- Combined LaTeX Table ---
\begin{table}[h!]
    \centering
    \caption{Side-by-Side Cumulative Forecast Results for Wells F-14 and F-12}
    \label{F-14_F-12_Cumulative_Combined}
    \resizebox{\textwidth}{!}{ %
      \begin{tabular}{@{}lllccc@{\quad}lllccc@{}}
        \toprule
        \multicolumn{6}{c}{\textbf{15/9-F-14}} & \multicolumn{6}{c}{\textbf{15/9-F-12}} \\
        \midrule
        \textbf{Strategy} & \textbf{EXTRACTOR} & \textbf{Fuser} & \textbf{R²} & \textbf{MAE} & \textbf{SMAPE} & \textbf{Strategy} & \textbf{EXTRACTOR} & \textbf{Fuser} & \textbf{R²} & \textbf{MAE} & \textbf{SMAPE} \\
        \midrule
Pressure Ensemble & AGGREGATE & Bias Scale & 1.00 & 3285.69 & 0.09 & Pressure Ensemble & AGGREGATE & Bias Scale & 1.00 & 3285.69 & 0.09 \\
Pressure Ensemble & IDENTITY & Concat & 1.00 & 4242.83 & 0.12 & Pressure Ensemble & IDENTITY & Concat & 1.00 & 4242.83 & 0.12 \\
Arps & AGGREGATE & Bias Scale & 1.00 & 4723.20 & 0.13 & Arps & AGGREGATE & Bias Scale & 1.00 & 4723.20 & 0